# Rewards Allocation Notebook
This goal of this notebook is to offer an easy way to process the outputs of the praise and sourcecred reward systems, calculate a reward distribution and perform an analysis of the results. It uses mock data and should be considered a work-in-progress. 

### Basic Setup
First, we import the relevant libraries, get the Data and set how many tokens we want to distribute

In [ ]:
from ipyfilechooser import FileChooser

import pandas as pd 
import numpy as np 
import analytics_toolbox as tb

import holoviews as hv
from holoviews import opts
import plotly.graph_objects as go
import plotly.express as pex

fc_praise = FileChooser('./exampleFiles')
fc_sourcecred = FileChooser('./exampleFiles')
fc_rewardboard = FileChooser('./exampleFiles')

print("== Please choose the Praise CSV file == ")
display(fc_praise)
print("== Please choose the Sourcecred CSV file == ")
display(fc_sourcecred)
print("== Please choose the Rewardboard address list CSV file == ")
display(fc_rewardboard)


Now that we have selected the files, we can import them for processing. We can also set the total amount of tokens we want to distribute this period.
Tip: Now that the file paths are set, you can safely click on "Cell > Run all below" from here on the menu bar to execute everything :) 

In [ ]:
PRAISE_DATA_PATH = fc_praise.selected
SOURCECRED_DATA_PATH = fc_sourcecred.selected
REWARD_BOARD_ADDRESSES_PATH = fc_rewardboard.selected

praise_data = pd.read_csv(PRAISE_DATA_PATH)
sourcecred_data = pd.read_csv(SOURCECRED_DATA_PATH)
rewardboard_addresses = pd.read_csv(REWARD_BOARD_ADDRESSES_PATH)

In [ ]:

NUMBER_OF_PRAISE_REWARD_TOKENS_TO_DISTRIBUTE = 1950
#Right now sourcecred rewards are calculated externally and already specified in the input. This may change.
NUMBER_OF_SOURCECRED_REWARD_TOKENS_TO_DISTRIBUTE = 1950
NUMBER_OF_REWARD_TOKENS_FOR_QUANTIFIERS = 1000
NUMBER_OF_REWARD_TOKENS_FOR_REWARD_BOARD = 100


## Praise reward allocation
This method allocates the praise rewards in a very straightforward way: It adds the value of all dished praised together, and then assigns to each user their % of the total.

In [ ]:
def calc_praise_rewards(praiseData, tokensToDistribute):
    #we discard all we don't need and and calculate the % worth of each praise
    slimData = praiseData[['FROM', 'TO', 'FINAL QUANT']].copy()
    totalPraisePoints = slimData['FINAL QUANT'].sum()

    slimData['PERCENTAGE'] = slimData['FINAL QUANT']/totalPraisePoints
    slimData['TOKEN TO RECEIVE'] = slimData['PERCENTAGE'] * tokensToDistribute
    return slimData

praise_distribution = calc_praise_rewards(praise_data.copy(), NUMBER_OF_PRAISE_REWARD_TOKENS_TO_DISTRIBUTE)
praise_distribution

#TO DO: show the whole table!

## Combining the Datasets

Now that we have the distribution done, we combine it with sourcecred into one table.
But before continuing, we also want to declare some methods for cleaning the data and prepare it. We'll use them later.

In [ ]:
#General Helper func. Puts all the "processing we probably won't need to do later or do differently" in one place
#  -removes the '#' and following from discord names
#  -Some renaming and dropping 
def prepare_praise(praise_data):
    praise_data['TO'] = (praise_data['TO'].str.split('#', 1, expand=False).str[0]).str.lower()
    praise_data.rename(columns = {'TO':'IDENTITY'}, inplace = True)
    praise_data = praise_data[['IDENTITY', 'PERCENTAGE', 'TOKEN TO RECEIVE']]
    return praise_data

#General Helper func. Puts all the "processing we probably won't need to do later or do differently" in one place
#  -Some renaming and dropping 
#  -changing percentages from 0 - 100 to 0.00-1.00
def prepare_sourcecred(sourcecred_data):
    sourcecred_data.rename(columns = {'%':'PERCENTAGE'}, inplace = True)
    sourcecred_data['IDENTITY'] = sourcecred_data['IDENTITY'].str.lower()
    sourcecred_data['PERCENTAGE'] = sourcecred_data['PERCENTAGE'] / 100
    sourcecred_data = sourcecred_data[['IDENTITY', 'PERCENTAGE', 'TOKEN TO RECEIVE']]
    return sourcecred_data

Now we are ready to go.

In [ ]:
#generates a new table with combined percentages and added token rewards
# TO DO adapt in case sourcecred stops sending the allocated token number
def combine_datasets(praise_data, sourcecred_data):
    processed_praise = prepare_praise(praise_data)
    processed_praise = processed_praise[['IDENTITY', 'PERCENTAGE', 'TOKEN TO RECEIVE']].groupby(['IDENTITY']).agg('sum').reset_index()
    processed_sourcecred = prepare_sourcecred(sourcecred_data)
    combined_dataset = processed_praise.append(processed_sourcecred, ignore_index=True)

    combined_dataset = combined_dataset.groupby(['IDENTITY']).agg('sum').reset_index()
    #since we just added to percentages
    combined_dataset['PERCENTAGE'] = combined_dataset['PERCENTAGE'] / 2


    return combined_dataset


total_period_rewards =combine_datasets(praise_distribution.copy(), sourcecred_data.copy())
total_period_rewards

![title](img/praiseFusion.jpg)

## Results Analysis
Let's dive into some data analysis! We'll use the metrics designed and explained by octopus🐙 for now, starting with:

### Allocation percentages
This table will show us which percentage if the total rewards gets distributed to which top % of users. So "Top 50% -> 0.85" would mean that the top 50% of praisees received 85% of the total rewards 

In [ ]:
p_vals = np.array([50,80,90,95,99])
rewards_rp = np.array([tb.resource_percentage(total_period_rewards["TOKEN TO RECEIVE"], p) for p in p_vals])

my_rd_index = [("Top " + str(100 - p) +"%") for p in p_vals]
resource_distribution = pd.DataFrame({"Rewards": rewards_rp}, index = my_rd_index)
print(resource_distribution)

### Gini coefficient
Next we will look at the Gini coefficient. Note that there is some debate if we want to use this metric at all, since it is usually employed to measure wealth distribution, and not compensation.

In [ ]:
p_vals = np.array([0, 50, 80])
rewards_gc = np.array([tb.gini_gt_p(np.array(total_period_rewards["TOKEN TO RECEIVE"]), p) for p in p_vals])

my_index = ["All", "Top 50%", "Top 20%"]
gini_coefs = pd.DataFrame({"Rewards": rewards_gc}, index = my_index)
print(gini_coefs)

### Shannon Entropy

[Shannon Entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) is a concept from communications theory, which is also used in measuring the diversity of a distribution. The formula for calculating Shannon Entropy among $n$ individuals is
    $$\\sum_{k=1}^n -p_k log_2(p_k),$$
where $p_k$ represents the proportion of the resource that user $k$ received.

Here we compare the actual Shannon Entropy with the maximum possible for the dataset, keeping in mind that a Shannon Entropy of 0 would mean one user holds all the rewards


In [ ]:
entropies_df = pd.DataFrame(data = {"Rewards" : tb.calc_shannon_entropies(total_period_rewards["PERCENTAGE"]) }, index = ["Entropy", "Max Entropy", "% of Max"])
print(entropies_df)

### Nakamoto Coefficient
Last but not least, the Nakamoto coefficient. The Nakamato Coefficient is defined as the smallest number of accounts who control at least 50% of the resource. Although its significance relates to the prospect of a 51% attack on a network, which may not be relevant in our context, we can still use it as an intuitive measure of how many individuals received the majority of a resource.

In [ ]:
ak_coef_IH = tb.nakamoto_coeff(total_period_rewards, "PERCENTAGE")
print(ak_coef_IH)

## Quantifier Data
### Praise by Quantifier
Let's take a closer look at each quantifier. In the following step we will use the raw praise data to zoom in on how each quantifier scored the praises:

In [ ]:
def data_by_quantifier(praise_data):
    quant_only = pd.DataFrame()
    praise_data.drop(['DATE', 'TO', 'FROM', 'REASON', 'SERVER', 'CHANNEL', 'CORRECTION ADD', 'CORRECTION SUB', 'CORRECTION COMMENT', 'FINAL QUANT'], axis=1, inplace=True)
    num_of_quants = int((praise_data.shape[1] -1)/ 4)
    for i in range(num_of_quants):
        q_name =  str( 'QUANT_'+ str(i+1) +'_ID' )
        q_value = str('QUANT_'+str(i+1) )
        buf = praise_data[['ID', q_name , q_value ]].copy()
    
        buf.rename(columns={q_name: 'QUANT_ID', q_value: 'QUANT_VALUE', 'ID':'PRAISE_ID'}, inplace=True)
        #print(buf)
        quant_only = quant_only.append(buf.copy(), ignore_index=True)

    columnsTitles = ['QUANT_ID', 'PRAISE_ID', 'QUANT_VALUE']
    quant_only.sort_values(['QUANT_ID', 'PRAISE_ID'], inplace=True)
    quant_only =  quant_only.reindex(columns=columnsTitles).reset_index(drop=True)
    return quant_only

quantifier_table = data_by_quantifier(praise_data.copy())
quantifier_table 



### Amount of praise quantified
With the above table we can easily see how much praise each quantifier rated.

In [ ]:
quant_praise_distribution = quantifier_table['QUANT_ID'].value_counts().reset_index().rename(columns={'index': 'QUANT_ADDRESS', 'QUANT_ID': 'NUMBER_OF_PRAISES'})
quant_praise_distribution

In [ ]:
fig = pex.pie(quant_praise_distribution, values='NUMBER_OF_PRAISES', names='QUANT_ADDRESS', title='Amount of praise by each quantifier')
fig.show()

### Praise Flows

We can also take a look at the top "praise flows". Thanks to @inventandchill for this awesome visualization! 
On one side we have the top 10 praise givers separately (modifiable by changing the variable n_senders), on the other the top 15 receivers (modifiable by changing the variable n_receivers). The people outside the selection get aggregated into the "REST FROM" and "REST TO" categories.

In [ ]:
hv.extension('bokeh')

In [ ]:
#n_senders: Left side. Praise senders. n largest ones + rest (others)
#n_receivers: Right side. Praise receivers. n largest ones + rest (others)
praise_flow = tb.prepare_praise_flow(praise_distribution.copy(), n_senders=10, n_receivers=15)
praise_flow

In [ ]:
%%opts Sankey (cmap='Category10' edge_color='FROM' edge_line_width=0 node_alpha=1.0)
%%opts Sankey [node_sort=False label_position='outer' bgcolor="snow" node_width=40 node_sort=True ]
%%opts Sankey [width=1200 height=1000 title="Praise flow for Batch 1. Sum of Praise. Left - praise sender. Right - praise receiver"]
%%opts Sankey [margin=0 padding=0 show_values=True]

hv.Sankey(praise_flow, kdims=["FROM", "TO"], vdims=["FINAL QUANT"])

### Histogram of Received Praise

Next we can see the distribution of the dished praise, i.e. "How many people had X amount of praise?"

In [ ]:

df_h=praise_distribution['FINAL QUANT'].copy()
#df_h.reset_index(inplace=True, drop = False)
frequencies, edges = np.histogram(praise_distribution['FINAL QUANT'], 200)
print('Values: %s, Edges: %s' % (frequencies.shape[0], edges.shape[0]))

#%%opts Histogram (cmap='Category10' edge_color='From' edge_line_width=0 node_alpha=1.0)
#%%opts Histogram [node_sort=False label_position='outer' bgcolor="snow" node_width=40 node_sort=True ]
#%%opts histogram [width=1200 height=1000 title="Praise flow for Batch 3. Sum of Praise. Left - praise sender. Right - praise receiver"]
#%%opts Sankey [margin=0 padding=0 show_values=True]
hv.Histogram((edges, frequencies)).opts(width=800, height=500, title='Praise amount histogram for Batch 1')

## Total Token Distribution Export
To send the allocations to the DAO for distribution, we need to put all data together and add the rewards for the reward board and the quantifiers. 

First, we will calculate the rewards for the Quantifiers and the Reward Board. This is fairly straightforward, since we distribute the allocated tokens equally.


In [ ]:
quantifier_rewards = pd.DataFrame(quant_praise_distribution['QUANT_ADDRESS'].copy())
quantifier_rewards['TOKEN TO RECEIVE'] = NUMBER_OF_REWARD_TOKENS_FOR_QUANTIFIERS / len(quantifier_rewards.index)
quantifier_rewards

In [ ]:
rewardboard_rewards = pd.DataFrame(rewardboard_addresses)
rewardboard_rewards['TOKEN TO RECEIVE'] = NUMBER_OF_REWARD_TOKENS_FOR_REWARD_BOARD / len(rewardboard_rewards.index)
rewardboard_rewards

Now we can merge them all into one table and save it, ready for distribution!

In [ ]:
## To Do: export it to csv
def prepare_export_data(total_period_rewards, quantifier_rewards, rewardboard_rewards):
    quantifier_rewards.rename(columns = {'QUANT_ADDRESS':'IDENTITY'}, inplace = True)
    rewardboard_rewards.rename(columns = {'ID':'IDENTITY'}, inplace = True)
    final_allocations = pd.DataFrame(total_period_rewards[['IDENTITY', 'TOKEN TO RECEIVE']])
    final_allocations = final_allocations.append(quantifier_rewards)
    final_allocations = final_allocations.append(rewardboard_rewards)
    final_allocations = final_allocations.reset_index(drop=True)
    return final_allocations

final_token_allocations = prepare_export_data(total_period_rewards, quantifier_rewards, rewardboard_rewards)
final_token_allocations